# Fruit Detection

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from time import time

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Train the model
train_data = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
valid_data = keras.preprocessing.image.ImageDataGenerator(1./255)
size = 100

# Training generators (classes will be automatically generated from subgategories)
train_generator = train_data.flow_from_directory('/Users/yschiele/Desktop/fruitsdata/Training/', target_size=(size, size),
                                                 batch_size=1, class_mode='categorical', color_mode="grayscale")
valid_generator = valid_data.flow_from_directory('/Users/yschiele/Desktop/fruitsdata/Test/', target_size=(size, size),
                                                 batch_size=1, class_mode='categorical', color_mode="grayscale")

train_data = train_generator.next()
test_data = valid_generator.next()

# print(np.squeeze(train_generator, axis=0).shape)



Found 3919 images belonging to 7 classes.
Found 1312 images belonging to 7 classes.


/Users/yschiele/anaconda2/lib/python2.7/site-packages/keras_preprocessing/image.py:1131: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [3]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(100, 100)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(7, activation=tf.nn.softmax)
])

model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

tensorboard = keras.callbacks.TensorBoard(log_dir="logs/{}".format(time()))

model.fit_generator(train_generator, epochs=5, steps_per_epoch=1, validation_data=valid_generator, validation_steps=7,
                    workers=4, callbacks=[tensorboard])

Epoch 1/5


ValueError: Error when checking input: expected flatten_input to have 3 dimensions, but got array with shape (1, 100, 100, 1)